# Exercise 1: Introduction to Machine Learning

Some code cells will be marked with 
```
##########################
######## To Do ###########
##########################
```

This indicates that you are being asked to write a piece of code to complete the notebook.

# Part 1: The Linear Classifier

To illustrate the workflow for training a deep learning model in a supervised manner, this notebook will walk you through the simple case of training a linear classifier to recognize images various stages of the cell cycle. While deep learning might seem intimidating, don't worry. Its conceptual underpinnings are rooted in linear algebra and calculus - if you can perform matrix multiplication and take derivatives you can understand what is happening in a deep learning workflow.

In [ ]:
import imageio as iio
import skimage
import sklearn.model_selection
import sklearn.utils
import sklearn.metrics
import skimage.color
import skimage.transform
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

## The supervised machine learning workflow
Recall from class the conceptual workflow for a supervised machine learning project. 
- First, we create a <em>training dataset</em>, a paired collection of raw data and labels where the labels contain information about the "insight" we wish to extract from the raw data. 
- Once we have training data, we can then use it to train a <em>model</em>. The model is a mathematical black box - it takes in data and transforms it into an output. The model has some parameters that we can adjust to change how it performs this mapping. 
- Adjusting these parameters to produce outputs that we want is called training the model. To do this we need two things. First, we need a notion of what we want the output to look like. This notion is captured by a <em>loss function</em>, which compares model outputs and labels and produces a score telling us if the model did a "good" job or not on our given task. By convention, low values of the loss function's output (e.g. the loss) correspond to good performance and high values to bad performance. We also need an <em>optimization algorithm</em>, which is a set of rules for how to adjust the model parameters to reduce the loss
- Using the training data, loss function, and optimization algorithm, we can then train the model 
- Once the model is trained, we need to evaluate its performance to see how well it performs and what kinds of mistakes it makes. We can also perform this kind of monitoring during training (this is actually a standard practice).

Because this workflow defines the lifecycle of most machine learning projects, this notebook is structured to go over each of these steps while constructing a linear classifier.


## Create training data
The starting point of every machine learning project is data. Today we are going to look at a collection of images of Jurkat cells published in the Broad Bioimage Collection ([BBBC048](https://bbbc.broadinstitute.org/BBBC048)). The cells were fixed and stained with PI (propidium iodide) to quantify DNA content and a MPM2 (mitotic protein monoclonal #2) antibody to identify mitotic cells.

During the initial setup of this exercise, we downloaded the data and unzipped the relevant files using the script `data-download.sh`.

In [ ]:
data_dir = 'data/CellCycle'
os.listdir(data_dir)

The command above should generate the following output. If you see something different, please check that the `data-download.sh` script ran correctly.
```
['img.lst~',
 'Anaphase',
 'Prophase',
 'img.lst',
 'S',
 '66.lst~',
 'G1',
 'Metaphase',
 'G2',
 'Telophase']
 ```
 
The metadata for each file is stored in `img.lst` so we will first load this information to inform how we load the rest of the dataset.

In [ ]:
# Load dataframe with sample info
df = pd.read_csv(os.path.join(data_dir, 'img.lst'), sep='\t', header=None)
df = df.rename(columns={1: 'class', 2: 'filepath'})
df['channel'] = df['filepath'].str.split('/',expand=True)[2].str.split('_', expand=True)[1].str.slice(2,3)
df['id'] = df['filepath'].str.split('/',expand=True)[2].str.split('_', expand=True)[0]
df.head()

In [ ]:
df['class'].unique()

For each `id` there are three images. One for each of the channels: phase, PI and MPM2. We will load each image and stack it into an array of training data.

In [ ]:
# Load data stack
ims = []
ys = []
for i, g in df.groupby('id'):
    im = []
    for _, r in g.iterrows():
        im.append(iio.imread(os.path.join(data_dir, r['filepath'])))
    ims.append(np.stack(im, axis=-1))
    ys.append(r['class'])
    
X_data = np.stack(ims)
y_data = np.stack(ys)
print('X shape:', X_data.shape)
print('y shape:', y_data.shape)

In the previous cell, you probably observed that there are 4 dimensions rather than the 3 you might have been expecting. This is because while each image is (66, 66, 3), the full dataset has many images. The different images are stacked along the first dimension. The full size of the training images is (# images, 66, 66, 3) - the first dimension is often called the batch dimension.

#### Task 1.1

In [ ]:
##########################
######## To Do ###########
##########################

# Use matplotlib (plt.imshow) to visualize several images randomly drawn from the dataset
# There are 6 classes in the dataset. Make sure to look at an example or each class


Let's check the balance of classes in this dataset. There are at least two ways you could do this. One would be to use matplotlib to create a histogram. The other would be to count the number of items in each class using numpy.

#### Task 1.2

In [ ]:
##########################
######## To Do ###########
##########################

# Add your code to check class balances here
# You should end up with a count of number of items in each of the 7 classes

This dataset is highly inbalanced so we will want to correct the class balance before training. We will resample the data after splitting it below.

Initially, we are going to restrict the dataset to two classes `[3, 5]` and a single channel `0`.

In [ ]:
def extract_classes(X_data, y_data, classes):
    X, y = [], []
    for c in classes:
        # Identify the indicies of the relevant class
        idx = y_data == c
        # Select the X and y data accordingly
        X.append(X_data[idx, ..., 0:1])
        y.append(y_data[idx])

    # Restack the arrays
    X = np.concatenate(X)
    y = np.concatenate(y)
    return X, y

In [ ]:
X, y = extract_classes(X_data, y_data, [3, 5])

print(X.shape, y.shape)

We will also reassign classes 3 and 5 to 0 and 1 for simplicity.

In [ ]:
y[y == 3] = 0
y[y == 5] = 1

For this exercise, we will want to flatten the training data into a vector

In [ ]:
# Flatten the images 1d vectors
X = np.reshape(X, (-1, 66 * 66, 1))
print(X.shape)

### Split the training dataset into training, validation, and testing datasets
How do we know how well our model is doing? A common practice to evaluate models is to evaluate them on splits of the original training dataset. Splitting the data is important, because we want to see how models perform on data that was not used to train them.
- The <em>training</em> dataset used to train the model
- A held out <em>testing</em> dataset used to evaluate the final trained version of the model
While there is no hard and fast rule, 80%, 20% splits are a reasonable starting point.

In [ ]:
# Split the dataset into training, validation, and testing splits
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=0.8, shuffle=True)
unbalanced = {
    'train': {'X': X_train, 'y': y_train},
    'test': {'X': X_test, 'y': y_test}
}

Now that the data is split into train and test splits we will upsample the minority class.

In [ ]:
def balance_classes(X, y):
    class_ids = np.unique(y)
    
    # Compute counts to determine minority class
    if np.count_nonzero(y == class_ids[0]) >= np.count_nonzero(y == class_ids[1]):
        max_id = 0
        min_id = 1
    else:
        max_id = 1
        min_id = 0
        
    # Split the classes into separate arrays
    x_max = X[y == max_id]
    y_max = y[y == max_id]
    x_min = X[y == min_id]
    y_min = y[y == min_id]
    
    # Upsample min class to match number of samples in max
    x_min, y_min = sklearn.utils.resample(x_min, y_min, n_samples=x_max.shape[0])
    
    # Concatenate the arrays from different classes back together 
    X = np.concatenate([x_max, x_min])
    y = np.concatenate([y_max, y_min])
    
    # Shuffle arrays to randomize sample order
    X, y = sklearn.utils.shuffle(X, y)
    
    return X, y

In [ ]:
balanced = {}
for split, data in unbalanced.items():
    xx, yy = balance_classes(data['X'], data['y'])

    # Save into the new dictionary
    balanced[split] = {'X': xx, 'y': yy}
    print(split, xx.shape, yy.shape)

## The linear classifier
The linear classifier produces class scores that are a linear function of the pixel values. Mathematically, this can be written as $\vec{y} = W \vec{x}$, where $\vec{y}$ is the vector of class scores, $W$ is a matrix of weights and $\vec{x}$ is the image vector. The shape of the weights matrix is determined by the number of classes and the length of the image vector. In this case $W$ is 2 by 4356. Our learning task is to find a set of weights that maximize our performance on our classification task. We will solve this task by doing the following steps
- Randomly initializing a set of weights
- Defining a loss function that measures our performance on the classification task
- Use stochastic gradient descent to find "optimal" weights

### Create the matrix of weights
Properly initializing weights is essential for getting deep learning methods to work correctly. The two most common initialization methods you'll see in this class are [glorot uniform (also known as Xavier) initialization](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf?hc_location=ufi]) and [he initialization](http://openaccess.thecvf.com/content_iccv_2015/html/He_Delving_Deep_into_ICCV_2015_paper.html) - both papers are worth reading. For this exercise, we will randomly initialize weights by using glorot uniform initialization. In this initialization method, we sample our weights according to the formula 
\begin{equation}
W_{ij} \sim U\left[ -\frac{1}{\sqrt{n}}, \frac{1}{\sqrt{n}} \right],
\end{equation}
where $n$ is the number of columns in the weight matrix (4,356 in our case).

Lets create the linear classifier using object oriented programming, which will help with organization

#### Task 1.3

In [ ]:
class LinearClassifier(object):
    def __init__(self, image_size=4356):
        self.image_size=image_size
        
        # Initialize weights
        self._initialize_weights()
        
    def _initialize_weights(self):
        
        ##########################
        ######## To Do ###########
        ##########################
        
        # Randomly initialize the weights matrix acccording to the glorot uniform initialization
        # self.W = # Add weights matrix here
        
        pass

### Apply the softmax transform to complete the model outputs
Our LinearClassifier class needs a method to perform predictions - which in our case is performing matrix multiplication and then applying the softmax transform. Recall from class that the softmax transform is given by
\begin{equation}
softmax(y_i) = \frac{e^{y_i}}{\sum_j e^{y_j}}
\end{equation}
and provides a convenient way to convert our class scores into probabilities.

Note that our predict function is going to start by removing the channel dimension from the X data.

#### Task 1.4

In [ ]:
##########################
######## To Do ###########
##########################

# Complete the predict function below to predict a label y from an input X
# Pay careful attention to the shape of your data at each step

def predict(self, X, epsilon=1e-5):
    X = X[..., 0]
    pass
    #y = # matrix multiplication

    #y = # Apply softmax
    return y

# Assign methods to class
setattr(LinearClassifier, 'predict', predict)

Now lets see what happens when we try to predict the class of images in our training dataset using randomly initialized weights.

In [ ]:
lc = LinearClassifier()

fig, axes = plt.subplots(4, 2, figsize=(20,20))
for i, j in enumerate(np.random.randint(balanced['test']['X'].shape[0], size=(8,))):
    
    # Get an example image
    X_sample = balanced['test']['X'][j,...]
    
    # Reshape flattened vector to image
    X_reshape = np.reshape(X_sample, (66, 66))
    
    # Predict the label
    y_pred = lc.predict(X_sample)
    
    # Display results
    axes.flatten()[i].imshow(X_reshape, cmap='gray')
    axes.flatten()[i].set_title('Label ' + str(balanced['test']['y'][j]) +', Prediction ' + str(np.argmax(y_pred)))

In addition to inspecting the results of individual predictions, we can also look at summary statistics that capture model performance.

In [ ]:
def benchmark_performance(y_true, y_pred):
    """Calculates recall, precision, f1 and a confusion matrix for sample predictions"""
    
    metrics = {
        'recall': sklearn.metrics.recall_score(y_true, y_pred),
        'precision': sklearn.metrics.precision_score(y_true, y_pred),
        'f1': sklearn.metrics.f1_score(y_true, y_pred),
        'cm': sklearn.metrics.confusion_matrix(y_true, y_pred, normalize='true'),
    }
    
    return metrics

#### Task 1.5
For each of the 4 metrics above, describe in your own words what this metric tells you about model performance.

########################  
######## To Do ###########  
########################  

- Recall
- Precision
- F1
- Confusion Matrix

In [ ]:
# Generate predictions and metrics for training data
y_pred = lc.predict(balanced['train']['X'])
y_pred = np.argmax(y_pred, axis=-1)
y_true = balanced['train']['y']

metrics = benchmark_performance(y_true, y_pred)

print('Training Recall: {}'.format(metrics['recall']))
print('Training Precision: {}'.format(metrics['precision']))
print('Training F1 Score: {}'.format(metrics['f1']),'\n')

# Visualize a confusion matrix
disp = sklearn.metrics.ConfusionMatrixDisplay(metrics['cm'])
disp.plot()
plt.show()

#### Task 1.6
What do you notice about the initial results of the model? 

########################  
######## To Do ###########  
########################  

## Stochastic gradient descent
To train this model, we will use stochastic gradient descent. In its simplest version, this algorithm consists of the following steps:
- Select several images from the training dataset at random
- Compute the gradient of the loss function with respect to the weights, given the selected images
- Update the weights using the update rule $\Delta W_{ij} \rightarrow \Delta W_{ij} - lr\frac{\partial loss}{\partial W_{ij}}$

Recall that the origin of this update rule is from multivariable calculus - the gradient tells us the direction in which the loss function increases the most. So to minimize the loss function we move in the opposite direction of the gradient.

Also recall from the course notes that for this problem we can compute the gradient analytically. The gradient is given by

\begin{equation}
\frac{\partial loss}{\partial W_{ij}} = \left(p_i - 1(i \mbox{ is correct}) \right)x_j,
\end{equation}
where $1$ is an indicator function that is 1 if the statement inside the parentheses is true and 0 if it is false.

In [ ]:
def grad(self, X, y):
    # Get class probabilities
    p = self.predict(X)
    
    # Compute class 0 gradients
    temp_0 = np.expand_dims(p[...,0] - (1-y), axis=-1)
    grad_0 = temp_0 * X[...,0]

    # Compute class 1 gradients
    temp_1 = np.expand_dims(p[...,1] - y, axis=-1)
    grad_1 =  temp_1 * X[...,0]
    
    gradient = np.stack([grad_0, grad_1], axis=1)
    
    return gradient
    
def loss(self, X, y_true):
    y_pred = self.predict(X)
    
    # Convert y_true to one hot
    y_true = np.stack([y_true, 1-y_true], axis=-1)
    loss = np.mean(-y_true * np.log(y_pred))
    
    return loss
    
def fit(self, X_train, y_train, n_epochs, batch_size=1, learning_rate=1e-5):
    # Iterate over epochs
    for epoch in range(n_epochs):
        n_batches = np.int(np.floor(X_train.shape[0] / batch_size))
        
        # Generate random index
        index = np.arange(X_train.shape[0])
        np.random.shuffle(index)
        
        # Iterate over batches
        loss_list = []
        for batch in range(n_batches):
            beg = batch*batch_size
            end = (batch+1)*batch_size if (batch+1)*batch_size < X_train.shape[0] else -1
            X_batch = X_train[beg:end]
            y_batch = y_train[beg:end]
            
            # Compute the loss
            loss = self.loss(X_batch, y_batch)
            loss_list.append(loss)
            
            # Compute the gradient
            gradient = self.grad(X_batch, y_batch)
            
            # Compute the mean gradient over all the example images
            gradient = np.mean(gradient, axis=0, keepdims=False)

            # Update the weights
            self.W -= learning_rate * gradient
            
        return loss_list

# Assign methods to class
setattr(LinearClassifier, 'grad', grad)
setattr(LinearClassifier, 'loss', loss)
setattr(LinearClassifier, 'fit', fit)

lc = LinearClassifier()
loss = lc.fit(balanced['train']['X'], balanced['train']['y'], n_epochs=32, batch_size=16)

## Evaluate the model
Benchmarking performance is a critical part of the model development process. For this problem, we will use 3 different benchmarks
- Recall: the fraction of positive examples detected by a model. Mathematically, for a two-class classification problem, recall is calculated as (True positives)/(True positives + False negatives). 
- Precision: the percentage of positive predictions from a model that are true. Mathematically, for a two-class prediction problem, precision is calculated as (True positives)/(True positives + False positives).
- F1 score: The harmonic mean between the recall and precision

We will evaluate these metrics on both the training dataset (the examples used during training) and our testing dataset (the examples that we held out). We can also use a confusion matrix to visualize the prediction results.

In [ ]:
# Visualize some predictions

fig, axes = plt.subplots(4, 2, figsize=(20,20))
for i, j in enumerate(np.random.randint(balanced['test']['X'].shape[0], size=(8,))):
    
    # Get an example image
    X_sample = balanced['test']['X'][j,...]
    
    # Reshape flattened vector to image
    X_reshape = np.reshape(X_sample, (66, 66))
    
    # Predict the label
    y_pred = lc.predict(X_sample)
    
    # Display results
    axes.flatten()[i].imshow(X_reshape, cmap='gray')
    axes.flatten()[i].set_title('Label ' + str(balanced['test']['y'][j]) +', Prediction ' + str(np.argmax(y_pred)))

In [ ]:
# Generate predictions and metrics for training data
y_pred = lc.predict(balanced['train']['X'])
y_pred = np.argmax(y_pred, axis=-1)
y_true = balanced['train']['y']

metrics = benchmark_performance(y_true, y_pred)

print('Training Recall: {}'.format(metrics['recall']))
print('Training Precision: {}'.format(metrics['precision']))
print('Training F1 Score: {}'.format(metrics['f1']),'\n')

# Visualize a confusion matrix
disp = sklearn.metrics.ConfusionMatrixDisplay(metrics['cm'])
disp.plot()
plt.show()

# Generate predictions and metrics for test data
y_pred = lc.predict(balanced['test']['X'])
y_pred = np.argmax(y_pred, axis=-1)
y_true = balanced['test']['y']

metrics = benchmark_performance(y_true, y_pred)

print('Testing Recall: {}'.format(metrics['recall']))
print('Testing Precision: {}'.format(metrics['precision']))
print('Testing F1 Score: {}'.format(metrics['f1']),'\n')

# Visualize a confusion matrix
disp = sklearn.metrics.ConfusionMatrixDisplay(metrics['cm'])
disp.plot()
plt.show()


# Part 2: Implementing a linear classifier with TensorFlow
In this section, we will define our machine learning models using TensorFlow. These models are composed of layers - each layer specifies a mathematical operation that is applied to its input. The nice thing about TensorFlow is that almost all of the machinery required for stochastic gradient descent is taken care of for us.
- Specify trainable variables? Check.
- Initialize trainable variables with random values? Check.
- Compute the layer outputs? Check.
- Compute gradients using backpropagation? Check.
- Perform all of the computations on GPUs to speed up training and inference? Check.
All of the above (and more) are taken care of for us by TensorFlow - writing models often requires little math (although one practice that I encourage is keeping track of the input and output dimensions for each layer).

To define a linear classifier, we will use a module in TensorFlow called Keras. Keras simple APIs for specifying models. In Keras, there are two different APIs you can use:
- [Sequential API](https://www.tensorflow.org/guide/keras/sequential_model) - If your model is composed of a linear sequence of steps, this is the easier API to use.
- [Functional API](https://www.tensorflow.org/guide/keras/functional) - If your model is more complicated, this API provides more flexibility. If you're using the functional API, consider using a class with methods to write submodels.
The TensorFlow documentation provides additional details about how to use each of these two APIs.

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
        
import tensorflow_addons as tfa

from tensorflow.keras.layers import Input, Flatten, Dense, Activation, BatchNormalization, Conv2D, MaxPool2D, Softmax
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model

## Create dataset object
TensorFlow uses Dataset objects to feed data into the training pipeline. These objects were covered in more detail in the TensorFlow Dataset notebook. In this section, we will make a class that builds a dataset object and applies random augmentation operation (e.g. rotation, flipping, scaling).

In [ ]:
# Create dataset builder
class DatasetBuilder(object):
    def __init__(self,
                 X,
                 y,
                 batch_size=1,
                 rotation_range=180,
                 scale_range=(0.75, 1.25)):
        self.X = X
        self.y = y
        
        self.batch_size = batch_size
        self.rotation_range = np.float(rotation_range)
        self.scale_range = scale_range
        
        # Create dataset
        self._create_dataset()
        
    def _augment(self, *args):
        img = args[0]
        label = args[1]
        
        theta = tf.random.uniform([1], 0, 2*np.pi*self.rotation_range/360)
        img = tfa.image.rotate(img, theta)
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        
        return (img, label)
        
    def _create_dataset(self):
        X_train, X_temp, y_train, y_temp = sklearn.model_selection.train_test_split(self.X, self.y, train_size=0.8)
        X_val, X_test, y_val, y_test = sklearn.model_selection.train_test_split(X_temp, y_temp, train_size=0.5)
        
        # Balance classes in each split
        X_train, y_train = balance_classes(X_train, y_train)
        X_test, y_test = balance_classes(X_test, y_test)
        X_val, y_val = balance_classes(X_val, y_val)
        
        # Convert y data to categorical
        y_train = tf.keras.utils.to_categorical(y_train)
        y_test = tf.keras.utils.to_categorical(y_test)
        y_val = tf.keras.utils.to_categorical(y_val)
        
        train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
        val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
        test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
        
        self.train_dataset = train_dataset.shuffle(256).batch(self.batch_size).map(self._augment)
        self.val_dataset = val_dataset.batch(self.batch_size)
        self.test_dataset = test_dataset.batch(self.batch_size)

In [ ]:
X, y = extract_classes(X_data, y_data, [3, 5])
y[y == 3] = 0
y[y == 5] = 1

with tf.device('CPU:0'):
    db = DatasetBuilder(X, y, batch_size=64)

In [ ]:
# Check the data augmentation
it = db.train_dataset.as_numpy_iterator()
X_temp, y_temp = it.next()
print(X_temp.shape)

fig, axes = plt.subplots(4, figsize=(20,20))
for i in range(4):
    axes.flatten()[i].imshow(X_temp[i,...,:], cmap='Greys')
    axes.flatten()[i].set_title('Label ' + str(y_temp[i]))

### Define the model

In [ ]:
# Define the linear classifier
def create_linear_classifier():
    inputs = Input((X.shape[1], X.shape[2], 1),
                   name='linear_classifier_input')
    x = Flatten()(inputs)
    x = Dense(2)(x)
    x = Softmax(axis=-1)(x)
    model = Model(inputs=inputs, outputs=x)
    return model

linear_classifier = create_linear_classifier()

## Specify training parameters 
In this section, we will specify how we want to train the neural network. We will need to specify three things:
- The loss function. Because we are training a model for classification, we will use the categorical crossentropy
- The training algorithm. There are many flavors of stochastic gradient descent - for this problem, we will use a variant called Adam
- The training parameters. The training algorithm needs parameters like the learning rate, number of epochs, number of steps per epoch, etc. to be specified

In [ ]:
# Define the loss function
loss_function = tf.keras.losses.CategoricalCrossentropy() 

In [ ]:
# Define the training algorithm
linear_optimizer = tf.keras.optimizers.Adam(lr=1e-3, clipnorm=0.001)
fc_optimizer = tf.keras.optimizers.Adam(lr=1e-3, clipnorm=0.001)
conv_optimizer = tf.keras.optimizers.Adam(lr=1e-3, clipnorm=0.001)

In [ ]:
# Define training parameters
training_steps_per_epoch=512
n_epochs=32

# Define callbacks
linear_model_path = 'linear'

linear_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        linear_model_path, monitor='val_loss',
        save_best_only=True, verbose=1,
        save_weights_only=False)
]

linear_callbacks.append(
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, verbose=1,
        patience=3, min_lr=1e-7)
)

# Define metrics
recall_0 = tf.keras.metrics.Recall(class_id=0)
recall_1 = tf.keras.metrics.Recall(class_id=1)

precision_0 = tf.keras.metrics.Precision(class_id=0)
precision_1 = tf.keras.metrics.Precision(class_id=1)

In [ ]:
# Compile models
linear_classifier.compile(optimizer=linear_optimizer, 
                          loss=loss_function, 
                          metrics = [recall_0, recall_1, precision_0, precision_1])

## Train the model
With the dataset, model, and training parameters defined, it is straightforward to train a model. Keras Model objects have a fit method that takes in the training parameters and executes the training algorithm.

In [ ]:
# Train the linear classifier
linear_classifier.fit(db.train_dataset,
                      validation_data=db.val_dataset,
                      epochs=n_epochs,
                      verbose=1,
                      callbacks=linear_callbacks)

## Benchmark the model
In this section, we will benchmark each model to assess the performance.

In [ ]:
# Visualize some predictions
it = db.test_dataset.as_numpy_iterator()

fig, axes = plt.subplots(4, 2, figsize=(20,20))
for i in range(8):
    X_test, y_test = it.next()
    
    # Get an example image
    X_sample = X_test[[i],...]
    
    # Predict the label
    y_pred_linear = linear_classifier.predict(X_sample)
    
    # Display results
    axes.flatten()[i].imshow(X_sample[0], cmap='gray')
    axes.flatten()[i].set_title('Label ' + str(np.argmax(y_test[i])) +', Prediction ' + str(y_pred[i]))

In [ ]:
# Generate predictions
test_list = list(db.test_dataset.as_numpy_iterator())
X_test = np.concatenate([item[0] for item in test_list], axis=0)
y_test = np.concatenate([item[1] for item in test_list], axis=0)
y_test = np.argmax(y_test, axis=-1)

# Compute linear classifier metrics
y_pred = linear_classifier.predict(X_test)
y_pred = np.argmax(y_pred, axis=-1)

metrics = benchmark_performance(y_test, y_pred)

print('Testing Recall: {}'.format(metrics['recall']))
print('Testing Precision: {}'.format(metrics['precision']))
print('Testing F1 Score: {}'.format(metrics['f1']),'\n')

# Visualize a confusion matrix
disp = sklearn.metrics.ConfusionMatrixDisplay(metrics['cm'])
disp.plot()
plt.show()

#### Task 2.1

Compare and contrast the performance of the first linear classifier that we trained to the version trained in tensorflow.

########################  
######## To Do ###########  
########################  

# Checkpoint

We will come back together as a group to discuss the exercise up to this point before moving on to the next notebook. If you have some extra time, try modifying the tensorflow model to work on 3 classes instead of just 2. At minimum, you will need to make the folloing modifications:
- Change the `balance_classes` function to work on more than 2 classes
- Update the initial input data to extract and assign 3 classes
- Modify `create_linear_classifier` to work on 3 classes